`신용거래 이상탐지 데이터 다루기`

#### set-up

In [ ]:
# 필요한 geopy 라이브러리를 설치

# %pip install geopy


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle

from geopy.distance import distance
from datetime import datetime

pd.set_option('display.max_columns', 50)

In [3]:
# 데이터 불러오기
# /Users/kwonkyoungmi/workspaces/workspace_Python/data/fraud.csv

df = pd.read_csv('/Users/kwonkyoungmi/workspaces/workspace_Python/data/fraud.csv')

## data checking

In [4]:
# cc_df의 컬럼명과 자료형을 확인합니다.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491134 entries, 0 to 491133
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   trans_date_trans_time  491134 non-null  object 
 1   cc_num                 491134 non-null  int64  
 2   merchant               491134 non-null  object 
 3   category               491134 non-null  object 
 4   amt                    491134 non-null  float64
 5   first                  491134 non-null  object 
 6   last                   491134 non-null  object 
 7   gender                 491134 non-null  object 
 8   street                 491134 non-null  object 
 9   city                   491134 non-null  object 
 10  state                  491134 non-null  object 
 11  zip                    491134 non-null  int64  
 12  lat                    491134 non-null  float64
 13  long                   491134 non-null  float64
 14  city_pop               491134 non-nu

| 컬럼명                  | 설명                                                                 | 데이터 타입 |
| :---------------------- | :------------------------------------------------------------------- | :---------- |
| `trans_date_trans_time` | 거래 날짜와 시간                                                       | `object`    |
| `cc_num`                | 신용 카드 번호                                                         | `int64`     |
| `merchant`              | 거래 가맹점 이름                                                       | `object`    |
| `category`              | 거래 유형                                                             | `object`    |
| `amt`                   | 거래 금액                                                             | `float64`   |
| `first`                 | 고객 이름                                                             | `object`    |
| `last`                  | 고객 성                                                              | `object`    |
| `gender`                | 고객 성별                                                             | `object`    |
| `street`                | 고객 주소                                                             | `object`    |
| `city`                  | 고객 거주 도시                                                         | `object`    |
| `state`                 | 고객 거주 주 (state)                                                  | `object`    |
| `zip`                   | 고객 우편번호                                                          | `int64`     |
| `lat`                   | 고객 위도                                                             | `float64`   |
| `long`                  | 고객 경도                                                             | `float64`   |
| `city_pop`              | 고객 거주 도시 인구수                                                  | `int64`     |
| `job`                   | 고객 직업                                                             | `object`    |
| `dob`                   | 고객 생년월일                                                          | `object`    |
| `trans_num`             | 거래 고유 번호                                                         | `object`    |
| `unix_time`             | 거래 시간 (Unix 타임스탬프)                                           | `int64`     |
| `merch_lat`             | 가맹점 위도                                                           | `float64`   |
| `merch_long`            | 가맹점 경도                                                           | `float64`   |
| `is_fraud`              | 사기 거래 여부 (1: 사기, 0: 정상)                                      | `int64`     |

In [5]:
# cc_df의 컬럼별 통계량을 확인합니다.
df.describe().round(1)

,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,4.911340e+05,491134.0,491134.0,491134.0,491134.0,491134.0,4.911340e+05,491134.0,491134.0,491134.0
mean,3.706013e+17,69.1,50770.5,37.9,-90.5,121392.2,1.358730e+09,37.9,-90.5,0.0
std,1.260229e+18,160.3,26854.9,5.3,13.0,372575.1,1.819402e+07,5.4,13.0,0.1
min,5.038744e+11,1.0,1843.0,24.7,-122.3,46.0,1.325376e+09,23.7,-123.3,0.0
25%,2.131124e+14,9.0,28405.0,33.7,-97.2,1228.0,1.343087e+09,33.8,-97.0,0.0
50%,3.531130e+15,42.2,49628.0,38.5,-87.6,5760.0,1.357257e+09,38.5,-87.6,0.0
75%,4.653879e+15,80.3,75048.0,41.5,-80.7,50835.0,1.374626e+09,41.6,-80.7,0.0
max,4.956829e+18,25086.9,99323.0,48.9,-70.0,2906700.0,1.388534e+09,49.9,-69.0,1.0


In [6]:
df.head(3)

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
1,2019-01-01 00:12:34,4956828990005111019,"fraud_Schultz, Simonis and Little",grocery_pos,44.71,Kenneth,Robinson,M,269 Sanchez Rapids,Elizabeth,NJ,7208,40.6747,-74.2239,124967,Operational researcher,1980-12-21,09eff9c806365e2a6be12c1bbab3d70e,1325376754,40.079588,-74.848087,0
2,2019-01-01 00:17:16,180048185037117,fraud_Kling-Grant,grocery_net,46.28,Mary,Wall,F,2481 Mills Lock,Plainfield,NJ,7060,40.6152,-74.4150,71485,Leisure centre manager,1974-07-19,19e23c6a300c774354417befe4f31f8c,1325377036,40.021888,-74.228188,0


## 불필요한 컬럼 제거하기

In [7]:
# 'merchant','job','cc_num' 컬럼에 포함된 값의 종류를 확인

x = ['merchant','job','cc_num']

for i in x:
    print(f'{i} : {df[i].nunique()}')

merchant : 693
job : 110
cc_num : 124


In [8]:
# 불필요한 컬럼들을 제거합니다.
# cc_df.drop(['merchant','first','last','street','city','state','zip','job','trans_num','unix_time'], axis = 1, inplace= True)

df_a = df.copy()
df_a.drop(['merchant','first','last','street','city','state','zip','job','trans_num','unix_time'], axis = 1, inplace= True)
df_a.head()

,trans_date_trans_time,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:44,630423337322,grocery_pos,107.23,F,48.8878,-118.2105,149,1978-06-21,49.159047,-118.186462,0
1,2019-01-01 00:12:34,4956828990005111019,grocery_pos,44.71,M,40.6747,-74.2239,124967,1980-12-21,40.079588,-74.848087,0
2,2019-01-01 00:17:16,180048185037117,grocery_net,46.28,F,40.6152,-74.4150,71485,1974-07-19,40.021888,-74.228188,0
3,2019-01-01 00:20:15,374930071163758,grocery_pos,64.09,M,42.2203,-83.3583,31515,1971-11-05,42.360426,-83.552316,0
4,2019-01-01 00:23:41,2712209726293386,misc_pos,25.58,F,30.4066,-91.1468,378909,1977-02-22,29.737426,-90.853194,0


In [9]:
# cc_num 컬럼의 값을 기준으로 정렬합니다.
df_a.sort_values(by = 'cc_num', inplace = True)

cols = ['cc_num'] + [col for col in df_a.columns if col != 'cc_num'] 
df_a = df_a[cols]
df_a.head()

,cc_num,trans_date_trans_time,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud
108802,503874407318,2019-07-05 23:23:58,entertainment,15.60,M,29.5894,-98.5201,1595797,1975-12-28,29.014544,-99.433890,0
324968,503874407318,2020-05-27 12:29:45,food_dining,48.70,M,29.5894,-98.5201,1595797,1975-12-28,30.074031,-98.790379,0
45703,503874407318,2019-04-01 05:26:03,grocery_net,44.71,M,29.5894,-98.5201,1595797,1975-12-28,29.851166,-98.934171,0
176940,503874407318,2019-10-12 17:01:30,entertainment,248.46,M,29.5894,-98.5201,1595797,1975-12-28,29.944531,-98.364703,0
265331,503874407318,2020-02-16 00:10:36,gas_transport,106.35,M,29.5894,-98.5201,1595797,1975-12-28,29.655990,-97.707776,0


## 구매금액의 z-score 구하기

In [52]:
# z-score 계산 실습

# 실습 데이터
temp = pd.DataFrame({'a': [10,20,30,20,10,200], 'b': [100,300,200,150,250,200], 'c': [10, 500, 20, 250, 25, 200]})

print(temp) ; print('\n')

temp.mean() ## temp 컬럼별 평균 계산
temp.std()  ## temp 컬럼별 표준편차 계산

# 각각의 데이터에 대해 각 컬럼의 z-score를 계산
# (data - mean) / std

for i in temp.columns:
    a = temp[i].mean()
    b = temp[i].std()

    print(f'{(temp[i] - a) / b}\n')

     a    b    c
0   10  100   10
1   20  300  500
2   30  200   20
3   20  150  250
4   10  250   25
5  200  200  200


0   -0.513321
1   -0.379411
2   -0.245501
3   -0.379411
4   -0.513321
5    2.030967
Name: a, dtype: float64

0   -1.414214
1    1.414214
2    0.000000
3   -0.707107
4    0.707107
5    0.000000
Name: b, dtype: float64

0   -0.818168
1    1.727244
2   -0.766221
3    0.428564
4   -0.740247
5    0.168828
Name: c, dtype: float64



In [11]:
# cc_num 컬럼의 값마다 데이터 개수를 계산
df_a['cc_num'].value_counts()

cc_num
30270432095985      4392
6538441737335434    4392
4642255475285942    4386
6538891242532018    4386
4364010865167176    4386
                    ... 
36913587729122      3641
4005676619255478    3638
4681601008538160    3638
30551643947183      3638
3511378610369890    3628
Name: count, Length: 124, dtype: int64

In [12]:
# cc_num 컬럼의 값마다 amt의 평균, 표준편차를 계산하여 amt_info에 저장합니다.
# /Users/kwonkyoungmi/workspaces/workspace_Python/data/

amt_info = df_a.groupby('cc_num')['amt'].agg(['mean','std']).reset_index()
amt_info.head()
amt_info.rename(columns = {'mean':'amt_mean', 'std':'amt_std'}, inplace = True)
amt_info.to_pickle('/Users/kwonkyoungmi/workspaces/workspace_Python/data/amt_info.pkl')

In [13]:
# Q. cc_num 컬럼을 기준으로, cc_df와 amt_info 데이터를 합쳐서 cc_df에 저장합니다.
# (left merge를 수행합니다.)

df_a = df_a.merge(amt_info, how = 'left', on = 'cc_num')


In [14]:
# Q. 결제금액(amt)의 z-score를 계산하여 amt_z 컬럼에 저장합니다.
# 평균: mean, 표준편차: std

df_a.apply(lambda x : (x['amt'] - x['amt_mean']) / x['amt_std'], axis = 1)

df_a['amt_z'] = df_a.apply(lambda x : (x['amt'] - x['amt_mean']) / x['amt_std'], axis = 1)
df_a.head()

,cc_num,trans_date_trans_time,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_mean,amt_std,amt_z
0,503874407318,2019-07-05 23:23:58,entertainment,15.60,M,29.5894,-98.5201,1595797,1975-12-28,29.014544,-99.433890,0,60.253406,127.265783,-0.350867
1,503874407318,2020-05-27 12:29:45,food_dining,48.70,M,29.5894,-98.5201,1595797,1975-12-28,30.074031,-98.790379,0,60.253406,127.265783,-0.090782
2,503874407318,2019-04-01 05:26:03,grocery_net,44.71,M,29.5894,-98.5201,1595797,1975-12-28,29.851166,-98.934171,0,60.253406,127.265783,-0.122133
3,503874407318,2019-10-12 17:01:30,entertainment,248.46,M,29.5894,-98.5201,1595797,1975-12-28,29.944531,-98.364703,0,60.253406,127.265783,1.478847
4,503874407318,2020-02-16 00:10:36,gas_transport,106.35,M,29.5894,-98.5201,1595797,1975-12-28,29.655990,-97.707776,0,60.253406,127.265783,0.362207


In [15]:
# 사기 거래 데이터(is_fraud=1)를 확인합니다.

df_a[df_a['is_fraud'] == 1]

,cc_num,trans_date_trans_time,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_mean,amt_std,amt_z
1883,503874407318,2020-01-04 23:14:10,food_dining,120.26,M,29.5894,-98.5201,1595797,1975-12-28,29.846302,-98.198533,1,60.253406,127.265783,0.471506
1915,503874407318,2020-01-04 22:46:55,shopping_net,1061.90,M,29.5894,-98.5201,1595797,1975-12-28,29.086806,-98.304101,1,60.253406,127.265783,7.870510
1948,503874407318,2020-01-04 22:12:50,shopping_net,918.91,M,29.5894,-98.5201,1595797,1975-12-28,28.698050,-97.534641,1,60.253406,127.265783,6.746956
2181,503874407318,2020-01-04 01:02:17,misc_net,777.16,M,29.5894,-98.5201,1595797,1975-12-28,29.117476,-97.574972,1,60.253406,127.265783,5.633145
2182,503874407318,2020-01-04 03:23:34,misc_net,832.04,M,29.5894,-98.5201,1595797,1975-12-28,30.000882,-98.160660,1,60.253406,127.265783,6.064368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491099,4956828990005111019,2019-10-24 22:40:16,shopping_net,1051.26,M,40.6747,-74.2239,124967,1980-12-21,41.574533,-74.383789,1,59.858059,132.138802,7.502731
491100,4956828990005111019,2019-10-24 02:45:02,misc_net,829.48,M,40.6747,-74.2239,124967,1980-12-21,41.506568,-75.022058,1,59.858059,132.138802,5.824345
491103,4956828990005111019,2019-10-24 01:53:32,misc_net,806.93,M,40.6747,-74.2239,124967,1980-12-21,40.742035,-74.244291,1,59.858059,132.138802,5.653691
491123,4956828990005111019,2019-10-23 23:00:50,shopping_net,914.36,M,40.6747,-74.2239,124967,1980-12-21,40.813549,-74.968022,1,59.858059,132.138802,6.466700


In [16]:
# z-score 계산이 완료되었으니 mean, std 컬럼을 제거합니다.

df_a.drop(['amt_mean','amt_std'], axis = 1, inplace = True)
df_a.head()

,cc_num,trans_date_trans_time,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z
0,503874407318,2019-07-05 23:23:58,entertainment,15.60,M,29.5894,-98.5201,1595797,1975-12-28,29.014544,-99.433890,0,-0.350867
1,503874407318,2020-05-27 12:29:45,food_dining,48.70,M,29.5894,-98.5201,1595797,1975-12-28,30.074031,-98.790379,0,-0.090782
2,503874407318,2019-04-01 05:26:03,grocery_net,44.71,M,29.5894,-98.5201,1595797,1975-12-28,29.851166,-98.934171,0,-0.122133
3,503874407318,2019-10-12 17:01:30,entertainment,248.46,M,29.5894,-98.5201,1595797,1975-12-28,29.944531,-98.364703,0,1.478847
4,503874407318,2020-02-16 00:10:36,gas_transport,106.35,M,29.5894,-98.5201,1595797,1975-12-28,29.655990,-97.707776,0,0.362207


In [17]:
# cc_num, category 컬럼의 값마다 결제금액(amt)의 평균, 표준편차를 계산하여 cat_info에 저장합니다.
# /Users/kwonkyoungmi/workspaces/workspace_Python/data/

cat_info = df_a.groupby(['cc_num','category'])['amt'].agg(['mean','std']).reset_index()
cat_info.head()
cat_info.rename(columns = {'mean':'cat_mean', 'std':'cat_std'}, inplace = True)
cat_info.to_pickle('/Users/kwonkyoungmi/workspaces/workspace_Python/data/cat_info.pkl')
cat_info.head()

,cc_num,category,cat_mean,cat_std
0,503874407318,entertainment,73.282418,103.050402
1,503874407318,food_dining,38.712305,46.548436
2,503874407318,gas_transport,68.457820,14.730440
3,503874407318,grocery_net,48.931302,18.736252
4,503874407318,grocery_pos,61.987806,23.449569


In [18]:
# Q. cc_num 컬럼을 기준으로, cc_df와 cat_info 데이터를 합쳐서 cc_df에 저장합니다.
# (left merge를 수행합니다.)

df_b = df_a.copy()
df_b = df_b.merge(cat_info, how = 'left', on = ['cc_num','category'])
df_b.head()

,cc_num,trans_date_trans_time,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_mean,cat_std
0,503874407318,2019-07-05 23:23:58,entertainment,15.60,M,29.5894,-98.5201,1595797,1975-12-28,29.014544,-99.433890,0,-0.350867,73.282418,103.050402
1,503874407318,2020-05-27 12:29:45,food_dining,48.70,M,29.5894,-98.5201,1595797,1975-12-28,30.074031,-98.790379,0,-0.090782,38.712305,46.548436
2,503874407318,2019-04-01 05:26:03,grocery_net,44.71,M,29.5894,-98.5201,1595797,1975-12-28,29.851166,-98.934171,0,-0.122133,48.931302,18.736252
3,503874407318,2019-10-12 17:01:30,entertainment,248.46,M,29.5894,-98.5201,1595797,1975-12-28,29.944531,-98.364703,0,1.478847,73.282418,103.050402
4,503874407318,2020-02-16 00:10:36,gas_transport,106.35,M,29.5894,-98.5201,1595797,1975-12-28,29.655990,-97.707776,0,0.362207,68.457820,14.730440


In [19]:
# Q. 결제금액(amt)의 z-score를 계산하여 cat_amt_z 컬럼에 저장합니다.
# 평균: mean, 표준편차: std

df_b.apply(lambda x : (x['amt'] - x['cat_mean']) / x['cat_std'], axis = 1)

df_b['cat_amt_z'] = df_b.apply(lambda x : (x['amt'] - x['cat_mean']) / x['cat_std'], axis = 1)
df_b.head()

,cc_num,trans_date_trans_time,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_mean,cat_std,cat_amt_z
0,503874407318,2019-07-05 23:23:58,entertainment,15.60,M,29.5894,-98.5201,1595797,1975-12-28,29.014544,-99.433890,0,-0.350867,73.282418,103.050402,-0.559750
1,503874407318,2020-05-27 12:29:45,food_dining,48.70,M,29.5894,-98.5201,1595797,1975-12-28,30.074031,-98.790379,0,-0.090782,38.712305,46.548436,0.214566
2,503874407318,2019-04-01 05:26:03,grocery_net,44.71,M,29.5894,-98.5201,1595797,1975-12-28,29.851166,-98.934171,0,-0.122133,48.931302,18.736252,-0.225301
3,503874407318,2019-10-12 17:01:30,entertainment,248.46,M,29.5894,-98.5201,1595797,1975-12-28,29.944531,-98.364703,0,1.478847,73.282418,103.050402,1.699921
4,503874407318,2020-02-16 00:10:36,gas_transport,106.35,M,29.5894,-98.5201,1595797,1975-12-28,29.655990,-97.707776,0,0.362207,68.457820,14.730440,2.572373


In [20]:
# z-score 계산이 완료되었으니 mean, std 컬럼을 제거합니다.

df_b.drop(['cat_mean','cat_std'], axis =1 , inplace = True)

## 결제 시간 관련 feature 분석

In [21]:
# Q. trans_date_trans_time 컬럼에서 시간 값을 추출하여 hour 컬럼에 저장합니다.
# (힌트: datetime 자료형을 이용합니다.)

df_b['trans_date_trans_time'] = pd.to_datetime(df_b['trans_date_trans_time'])
df_b['trans_date_trans_time'].dt.hour
df_b['hour'] = df_b['trans_date_trans_time'].dt.hour
df_b.head()
# [[YOUR CODE]]

,cc_num,trans_date_trans_time,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour
0,503874407318,2019-07-05 23:23:58,entertainment,15.60,M,29.5894,-98.5201,1595797,1975-12-28,29.014544,-99.433890,0,-0.350867,-0.559750,23
1,503874407318,2020-05-27 12:29:45,food_dining,48.70,M,29.5894,-98.5201,1595797,1975-12-28,30.074031,-98.790379,0,-0.090782,0.214566,12
2,503874407318,2019-04-01 05:26:03,grocery_net,44.71,M,29.5894,-98.5201,1595797,1975-12-28,29.851166,-98.934171,0,-0.122133,-0.225301,5
3,503874407318,2019-10-12 17:01:30,entertainment,248.46,M,29.5894,-98.5201,1595797,1975-12-28,29.944531,-98.364703,0,1.478847,1.699921,17
4,503874407318,2020-02-16 00:10:36,gas_transport,106.35,M,29.5894,-98.5201,1595797,1975-12-28,29.655990,-97.707776,0,0.362207,2.572373,0


In [22]:
# 결제시간을 morning, afternoon, night, evening으로 분류하기 위해 함수를 정의합니다.

def hour_func(x):
    if (x >= 6) & (x < 12):
        return 'morning'
    elif (x >= 12) & (x < 18):
        return 'afternoon'
    elif (x >= 18) & (x < 23):
        return 'night'
    else:
        return 'evening'
    
df_b['hour'].apply(hour_func)

0           evening
1         afternoon
2           evening
3         afternoon
4           evening
            ...    
491129    afternoon
491130      morning
491131      evening
491132      evening
491133      morning
Name: hour, Length: 491134, dtype: object

In [23]:
# Q. hour 컬럼에 hour_func 함수를 적용한 값을 hour_cat 컬럼에 저장합니다.

df_b['hour_cat'] = df_b['hour'].apply(hour_func)
df_b.head()

,cc_num,trans_date_trans_time,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour,hour_cat
0,503874407318,2019-07-05 23:23:58,entertainment,15.60,M,29.5894,-98.5201,1595797,1975-12-28,29.014544,-99.433890,0,-0.350867,-0.559750,23,evening
1,503874407318,2020-05-27 12:29:45,food_dining,48.70,M,29.5894,-98.5201,1595797,1975-12-28,30.074031,-98.790379,0,-0.090782,0.214566,12,afternoon
2,503874407318,2019-04-01 05:26:03,grocery_net,44.71,M,29.5894,-98.5201,1595797,1975-12-28,29.851166,-98.934171,0,-0.122133,-0.225301,5,evening
3,503874407318,2019-10-12 17:01:30,entertainment,248.46,M,29.5894,-98.5201,1595797,1975-12-28,29.944531,-98.364703,0,1.478847,1.699921,17,afternoon
4,503874407318,2020-02-16 00:10:36,gas_transport,106.35,M,29.5894,-98.5201,1595797,1975-12-28,29.655990,-97.707776,0,0.362207,2.572373,0,evening


In [24]:
# Q. 시간대별로 데이터 수를 계산하여 출력합니다.
df_b['hour_cat'].value_counts()

hour_cat
afternoon    176801
night        146697
evening       98662
morning       68974
Name: count, dtype: int64

In [25]:
# cc_num 컬럼의 값마다 amt 컬럼의 데이터 수를 계산하여 all_cnt 변수로 저장합니다.

all_cnt = df_b.groupby('cc_num')['amt'].count().reset_index()

all_cnt.head()
all_cnt.rename(columns = {'amt':'amt_cnt_byCCnum'}, inplace = True)
all_cnt.head()

,cc_num,amt_cnt_byCCnum
0,503874407318,3655
1,567868110212,3644
2,571365235126,4374
3,581686439828,3653
4,630423337322,4362


In [26]:
# cc_num, hour_cat 컬럼의 값마다 amt 컬럼의 데이터 수를 계산하여 hour_cnt 변수로 저장합니다.

hour_cnt = df_b.groupby(['cc_num','hour_cat'])['amt'].count().reset_index()
hour_cnt.head()
hour_cnt.rename(columns = {'amt':'amt_cnt_byCCnumHourCat'}, inplace = True)

In [27]:
# Q. cc_num 컬럼을 기준으로 hour_cnt와 all_cnt를 합칩니다.
# (hour_cnt에서 left merge를 수행합니다.)

hour_cnt.merge(all_cnt, how = 'left', on = 'cc_num').head()

,cc_num,hour_cat,amt_cnt_byCCnumHourCat,amt_cnt_byCCnum
0,503874407318,afternoon,1280,3655
1,503874407318,evening,737,3655
2,503874407318,morning,558,3655
3,503874407318,night,1080,3655
4,567868110212,afternoon,1228,3644


In [28]:
# Q. amt_x와 amt_y 컬럼의 이름을 각각 hour_cnt, total_cnt로 변경합니다.

hour_cnt_m = hour_cnt.merge(all_cnt, how = 'left', on = 'cc_num').rename(columns = {'amt_cnt_byCCnumHourCat':'hour_cnt','amt_cnt_byCCnum':'total_cnt'})
hour_cnt_m.head()

,cc_num,hour_cat,hour_cnt,total_cnt
0,503874407318,afternoon,1280,3655
1,503874407318,evening,737,3655
2,503874407318,morning,558,3655
3,503874407318,night,1080,3655
4,567868110212,afternoon,1228,3644


In [29]:
# Q. 전체 거래 건수(total_cnt)에 대한 시간대별 거래 건수(hour_cnt)의 비율을 계산하여 
# hour_perc 컬럼에 저장합니다.

hour_cnt_m['hour_perc'] = hour_cnt_m['hour_cnt'] / hour_cnt_m['total_cnt']
hour_cnt_m.head()

,cc_num,hour_cat,hour_cnt,total_cnt,hour_perc
0,503874407318,afternoon,1280,3655,0.350205
1,503874407318,evening,737,3655,0.201642
2,503874407318,morning,558,3655,0.152668
3,503874407318,night,1080,3655,0.295486
4,567868110212,afternoon,1228,3644,0.336992


In [30]:
# # /Users/kwonkyoungmi/workspaces/workspace_Python/data/
hour_cnt = hour_cnt_m[['cc_num','hour_cat','hour_perc']]
hour_cnt.to_pickle('/Users/kwonkyoungmi/workspaces/workspace_Python/data/hour_cnt.pkl')

In [31]:
# Q. cc_num, hour_cat 컬럼을 기준으로 cc_df와 hour_cnt 데이터를 합칩니다.
# (cc_df에서 left merge를 수행합니다.)

df_c = df_b.copy()
df_c = df_c.merge(hour_cnt, how = 'left', on = ['cc_num','hour_cat'])
df_c.head()


,cc_num,trans_date_trans_time,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour,hour_cat,hour_perc
0,503874407318,2019-07-05 23:23:58,entertainment,15.60,M,29.5894,-98.5201,1595797,1975-12-28,29.014544,-99.433890,0,-0.350867,-0.559750,23,evening,0.201642
1,503874407318,2020-05-27 12:29:45,food_dining,48.70,M,29.5894,-98.5201,1595797,1975-12-28,30.074031,-98.790379,0,-0.090782,0.214566,12,afternoon,0.350205
2,503874407318,2019-04-01 05:26:03,grocery_net,44.71,M,29.5894,-98.5201,1595797,1975-12-28,29.851166,-98.934171,0,-0.122133,-0.225301,5,evening,0.201642
3,503874407318,2019-10-12 17:01:30,entertainment,248.46,M,29.5894,-98.5201,1595797,1975-12-28,29.944531,-98.364703,0,1.478847,1.699921,17,afternoon,0.350205
4,503874407318,2020-02-16 00:10:36,gas_transport,106.35,M,29.5894,-98.5201,1595797,1975-12-28,29.655990,-97.707776,0,0.362207,2.572373,0,evening,0.201642


In [32]:
# 시간 관련 feature 분석이 완료되었으므로 불필요해진 컬럼들을 제거합니다.

df_c.drop(['trans_date_trans_time', 'hour', 'hour_cat'], axis =1 , inplace = True)

## 거리 관련 feature 분석

In [33]:
# 고객의 위치와 상점의 위치 사이의 거리를 계산하여 distance 컬럼에 저장합니다.
# (모든 데이터가 계산되므로 실행 시간이 오래 걸릴 수 있습니다.)

df_c['distance'] = df_c.apply(lambda x: distance((x['lat'], x['long']), (x['merch_lat'], x['merch_long'])).km, axis = 1)
df_c.head()

,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour_perc,distance
0,503874407318,entertainment,15.60,M,29.5894,-98.5201,1595797,1975-12-28,29.014544,-99.433890,0,-0.350867,-0.559750,0.201642,109.276725
1,503874407318,food_dining,48.70,M,29.5894,-98.5201,1595797,1975-12-28,30.074031,-98.790379,0,-0.090782,0.214566,0.350205,59.735407
2,503874407318,grocery_net,44.71,M,29.5894,-98.5201,1595797,1975-12-28,29.851166,-98.934171,0,-0.122133,-0.225301,0.201642,49.467504
3,503874407318,entertainment,248.46,M,29.5894,-98.5201,1595797,1975-12-28,29.944531,-98.364703,0,1.478847,1.699921,0.350205,42.136908
4,503874407318,gas_transport,106.35,M,29.5894,-98.5201,1595797,1975-12-28,29.655990,-97.707776,0,0.362207,2.572373,0.201642,79.018246


In [34]:
# 데이터 10000개만 계산해도 시간이 이렇게 걸립니다.

start_time =  datetime.now()
df_c.head(10000).apply(lambda x: distance((x['lat'], x['long']), (x['merch_lat'], x['merch_long'])).km, axis = 1)
datetime.now() - start_time

datetime.timedelta(seconds=2, microseconds=751262)

In [40]:
# Q. cc_num 컬럼의 값마다 distance의 평균, 표준편차를 계산하여 dist_info 변수에 저장합니다.

dist_info = df_c.groupby('cc_num')['distance'].agg(['mean','std']).reset_index()

dist_info.rename(columns = {'mean':'dist_mean', 'std':'dist_std'}, inplace = True)
# /Users/kwonkyoungmi/workspaces/workspace_Python/data/
dist_info.to_pickle('/Users/kwonkyoungmi/workspaces/workspace_Python/data/dist_info.pkl')


In [41]:
# Q. cc_num 컬럼을 기준으로 cc_df와 dist_info 데이터를 합칩니다.
# (cc_df에서 left merge를 수행합니다.)

df_c = df_c.merge(dist_info, how = 'left', on = 'cc_num')
df_c.head()


,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour_perc,distance,dist_mean,dist_std
0,503874407318,entertainment,15.60,M,29.5894,-98.5201,1595797,1975-12-28,29.014544,-99.433890,0,-0.350867,-0.559750,0.201642,109.276725,79.746738,29.741044
1,503874407318,food_dining,48.70,M,29.5894,-98.5201,1595797,1975-12-28,30.074031,-98.790379,0,-0.090782,0.214566,0.350205,59.735407,79.746738,29.741044
2,503874407318,grocery_net,44.71,M,29.5894,-98.5201,1595797,1975-12-28,29.851166,-98.934171,0,-0.122133,-0.225301,0.201642,49.467504,79.746738,29.741044
3,503874407318,entertainment,248.46,M,29.5894,-98.5201,1595797,1975-12-28,29.944531,-98.364703,0,1.478847,1.699921,0.350205,42.136908,79.746738,29.741044
4,503874407318,gas_transport,106.35,M,29.5894,-98.5201,1595797,1975-12-28,29.655990,-97.707776,0,0.362207,2.572373,0.201642,79.018246,79.746738,29.741044


In [43]:
# Q. distance 값의 z-score를 계산하여 dist_z 컬럼에 저장합니다.

df_f = df_c.copy()

df_f['dist_z'] = df_f.apply(lambda x: (x['distance']- x['dist_mean'])/ x['dist_std'], axis = 1)
df_f.head()

,cc_num,category,amt,gender,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud,amt_z,cat_amt_z,hour_perc,distance,dist_mean,dist_std,dist_z
0,503874407318,entertainment,15.60,M,29.5894,-98.5201,1595797,1975-12-28,29.014544,-99.433890,0,-0.350867,-0.559750,0.201642,109.276725,79.746738,29.741044,0.992903
1,503874407318,food_dining,48.70,M,29.5894,-98.5201,1595797,1975-12-28,30.074031,-98.790379,0,-0.090782,0.214566,0.350205,59.735407,79.746738,29.741044,-0.672852
2,503874407318,grocery_net,44.71,M,29.5894,-98.5201,1595797,1975-12-28,29.851166,-98.934171,0,-0.122133,-0.225301,0.201642,49.467504,79.746738,29.741044,-1.018096
3,503874407318,entertainment,248.46,M,29.5894,-98.5201,1595797,1975-12-28,29.944531,-98.364703,0,1.478847,1.699921,0.350205,42.136908,79.746738,29.741044,-1.264577
4,503874407318,gas_transport,106.35,M,29.5894,-98.5201,1595797,1975-12-28,29.655990,-97.707776,0,0.362207,2.572373,0.201642,79.018246,79.746738,29.741044,-0.024495


In [44]:
# 불필요해진 컬럼들을 제거합니다.

df_f.drop(['lat','long','merch_lat','merch_long','dist_mean','dist_std'], axis = 1, inplace = True)
df_f.head()

,cc_num,category,amt,gender,city_pop,dob,is_fraud,amt_z,cat_amt_z,hour_perc,distance,dist_z
0,503874407318,entertainment,15.60,M,1595797,1975-12-28,0,-0.350867,-0.559750,0.201642,109.276725,0.992903
1,503874407318,food_dining,48.70,M,1595797,1975-12-28,0,-0.090782,0.214566,0.350205,59.735407,-0.672852
2,503874407318,grocery_net,44.71,M,1595797,1975-12-28,0,-0.122133,-0.225301,0.201642,49.467504,-1.018096
3,503874407318,entertainment,248.46,M,1595797,1975-12-28,0,1.478847,1.699921,0.350205,42.136908,-1.264577
4,503874407318,gas_transport,106.35,M,1595797,1975-12-28,0,0.362207,2.572373,0.201642,79.018246,-0.024495


## 나이 feature 만들어 보기

In [45]:
# Q. dob 컬럼에서 연도 값만 추출하여 dob 컬럼에 저장합니다.

df_f['dob'] = pd.to_datetime(df_f['dob']).dt.year
df_f.head()

,cc_num,category,amt,gender,city_pop,dob,is_fraud,amt_z,cat_amt_z,hour_perc,distance,dist_z
0,503874407318,entertainment,15.60,M,1595797,1975,0,-0.350867,-0.559750,0.201642,109.276725,0.992903
1,503874407318,food_dining,48.70,M,1595797,1975,0,-0.090782,0.214566,0.350205,59.735407,-0.672852
2,503874407318,grocery_net,44.71,M,1595797,1975,0,-0.122133,-0.225301,0.201642,49.467504,-1.018096
3,503874407318,entertainment,248.46,M,1595797,1975,0,1.478847,1.699921,0.350205,42.136908,-1.264577
4,503874407318,gas_transport,106.35,M,1595797,1975,0,0.362207,2.572373,0.201642,79.018246,-0.024495


## 범주형 데이터의 ont-hot encoding

In [46]:
# 범주형 컬럼 'category'에 몇 종류의 값이 있는지 확인해봅시다.

df_f['category'].nunique()

14

In [48]:
# Q. cc_df의 범주형 데이터에 원-핫 인코딩을 적용합니다.
# (drop_first 옵션은 True로 설정합니다.)
df_g = df_f.copy()
df_g = pd.get_dummies(df_f, drop_first = True)
df_g.head()

,cc_num,amt,city_pop,dob,is_fraud,amt_z,cat_amt_z,hour_perc,distance,dist_z,category_food_dining,category_gas_transport,category_grocery_net,category_grocery_pos,category_health_fitness,category_home,category_kids_pets,category_misc_net,category_misc_pos,category_personal_care,category_shopping_net,category_shopping_pos,category_travel,gender_M
0,503874407318,15.60,1595797,1975,0,-0.350867,-0.559750,0.201642,109.276725,0.992903,False,False,False,False,False,False,False,False,False,False,False,False,False,True
1,503874407318,48.70,1595797,1975,0,-0.090782,0.214566,0.350205,59.735407,-0.672852,True,False,False,False,False,False,False,False,False,False,False,False,False,True
2,503874407318,44.71,1595797,1975,0,-0.122133,-0.225301,0.201642,49.467504,-1.018096,False,False,True,False,False,False,False,False,False,False,False,False,False,True
3,503874407318,248.46,1595797,1975,0,1.478847,1.699921,0.350205,42.136908,-1.264577,False,False,False,False,False,False,False,False,False,False,False,False,False,True
4,503874407318,106.35,1595797,1975,0,0.362207,2.572373,0.201642,79.018246,-0.024495,False,True,False,False,False,False,False,False,False,False,False,False,False,True


In [49]:
# 이제 불필요해진 cc_num 컬럼을 제거합니다.

df_g.drop('cc_num', axis = 1, inplace = True)